In [199]:
import python_files.functions_barcelona as fb
import datetime
import math
import pickle
import pandas as pd
import geopandas as gpd
import difflib
import requests
import json
import os
import numpy as np
from datetime import date, datetime, timedelta
#from python_files.functions_barcelona import getting_daily_weather,getting_next_day
#rom python_files.functions_barcelona import concatenating_dataframes
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

import webbrowser
%load_ext autoreload
%autoreload 2
print('DONE')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DONE


In [200]:
###ORIGINAL WORKING

response = requests.get('https://opendata-ajuntament.barcelona.cat/data/api/3/action/package_search?rows=1000').json()

noms=['causes-', 'persones-','tipus-', 'vehicles-','','causa_conductor_']
##MAYBE ADDING CAUSA_CONDUCTOR
dict_files={}
for nom in noms:
   
    dict_files[nom[:-1]]=fb.concatenating_dataframes(nom,response)
    print(nom[:-1] +' DONE')

with open("./data/dataframes_dict_files.pkl", "wb" ) as f:
    pickle.dump( dict_files, f)
print("Done")

causes DONE
persones DONE
tipus DONE
vehicles DONE
 DONE
causa_conductor DONE
Done


In [227]:
last_year=input("What is the last year we are doing?")
with open("./data/dataframes_dict_files.pkl", "rb" ) as file:
    dict_files=pickle.load(file)
columnes=['num_incident', 'district_code','district', 'neighborhood','street_name',
          'weekday', 'year', 'month', 'day', 'hour', 'ped_cause',
           'num_deaths', 'num_minorly_injured', 'num_severly_injured',
            'num_victims', 'num_vehicles', 'utm_y', 'utm_x']
replacements={"Numero_expedient": "num_incident",
              "Numero_d_expedient": "num_incident",
              "Numero_dexpedient": "num_incident",
              'N£mero_dexpedient': "num_incident",
              'Numero_Expedient': "num_incident",
              'Codi_expedient': 'num_incident',
              'Codi_dexpedient':'num_incident',
                "Nom_districte": "district",
              'Codi_districte': 'district_code',
              'Codi districte': 'district_code',
               "Nom_barri":"neighborhood",
              "Nom_carrer":"street_name",
              "Descripcio_dia_setmana":"weekday",
              "NK_Any":"year",
              "Any":"year",
              "Nom_mes":"month",
              "Dia_mes":"day",
              "Dia_de_mes":"day",
              "Hora_dia":"hour",
              "Descripcio_causa_vianant":"ped_cause",
              "Numero_morts":"num_deaths",
              "Numero_lesionats_lleus":"num_minorly_injured",
              "Numero_lesionats_greus":"num_severly_injured",
              "Numero_victimes":"num_victims",
              "Numero_vehicles_implicats":"num_vehicles",
              "Coordenada_UTM_X_ED50":"utm_x",
              "Coordenada_UTM_Y_ED50":"utm_y",
            "Coordenada_UTM_X":"utm_x",
             "Coordenada_UTM_Y":"utm_y",
             "Coordenada UTM (Y)": "utm_y",
             "Coordenada UTM (X)": "utm_x",
             "Coordenada_UTM_(Y)":"utm_y",
             "Coordenada_UTM_(X)":"utm_x",
             "Longitud_WGS84":"Longitud",
             "Latitud_WGS84":"Latitud",
             'Num_postal_':"Num_postal",
             "Num_postal_caption":"Num_postal",
             'Tipus_accident':'accident_type',
              'Descripcio_tipus_accident':'accident_type',
              'Descripcio_color':'vehicle_color',
              'Descripcio_model':'vehicle_model',
              'Descripcio_marca':'vehicle_brand_name',
              'Descripcio_carnet': 'license',
              'Antiguitat_carnet':'license_seniority'
              
             }

llista_=['causes', 'persones', 'tipus', 'vehicles', '']
for grup in llista_:
    for num, df in enumerate(dict_files[grup]):
        df.columns=[col.replace("ú",'u').replace("ó","o").replace("'",'').replace(' ','_').replace("_de_","_").replace("í","i").replace("¢",'o') for col in df]

        df=df.rename(columns=replacements,)
        dict_files[grup][num]=df
#acc=pd.DataFrame(columns=columnes)
df_list=[]   
for dd in dict_files['']:
    columns=[col for col in dd.columns if col in columnes]
    dd_final=dd[columns]
    df_list.append(dd_final)
    #acc=pd.concat([acc,dd_final])
    #print(acc.shape)
acc=pd.concat(df_list)

##Fixing utm_x and utm_y that are mixed in some cases. Replacing nulls(-1) with the mean
acc['utm_x']=[x[0] if len(str(x[0]).split('.')[0])==7 else x[1] for x in zip(acc.utm_x,acc.utm_y)]
acc['utm_y']=[x[1] if len(str(x[1]).split('.')[0])==6 else x[0] for x in zip(acc.utm_x,acc.utm_y)]

##Fixing other stuff
acc['ped_cause']=acc['ped_cause'].apply(fb.ped_to_angles)
acc['weekday']=acc['weekday'].apply(fb.setmana_a_angles)
acc['month']=acc['month'].apply(fb.mes_a_angles)
acc['num_incident']=[x.strip() for x in acc['num_incident']]
#print(acc.shape)
acc=acc.drop_duplicates(subset='num_incident',keep='last')

print(acc.shape)


dups=acc[acc['num_incident'].duplicated(keep=False)]
dups[dups[['num_incident','ped_cause']].duplicated()]
##all duplicates are due to have 2 different ped cause being the rest exactly the same. Total dups: 27. I will just keep the first. No big deal; does not pay wasting time
acc.fillna(0,inplace=True)
print(acc.isnull().sum())

acc=acc.dropna()
print('EL BO:',acc.shape)
acc['district_code']=[str(int(x)) if x <0 or x>9 else '0'+str(int(x)) for x in acc.district_code]
acc.to_csv(f'./data/accidents_only{last_year}.csv',index=False)
print(acc.isnull().sum())

print("DONE WITH ACCIDENTS")


#CAUSES

for num, df in enumerate(dict_files['causes']):
    columns_to_save=[]
    for col in df:
        if ('incident' in col.lower()) or ('mediata' in col.lower()):
            columns_to_save.append(col)
    
    dict_files['causes'][num]=df[columns_to_save]
    dict_files['causes'][num].columns=['num_incident' if 'incident' in col else 'cause' for col in columns_to_save]
    
causes=pd.concat(dict_files['causes']).reset_index(drop=True)
causes['num_incident']=[x.strip() for x in causes['num_incident']]
causes['cause']=causes.cause.apply(fb.posant_accents).apply(fb.cause_to_angles)
causes=causes.drop_duplicates('num_incident',keep='last')
causes.to_csv(f'./data/causes{last_year}.csv')
print(acc.shape,causes.shape)
total= pd.merge(acc,causes, how='inner',on='num_incident')
print('TOTAL',total.shape)
print("DONE WITH CAUSES")

#descripcio situacio: no interessa, no te valor predictiu
# No interest either on 'Descripcio_victimitzacio','Descripcio_Motiu_desplaçament_vianant','Descripcio_Motiu_desplaçament_conductor'
chosen_features=['num_incident','Descripcio_sexe','Edat','Descripcio_tipus_persona','Descripcio_tipus_persona','Desc_Tipus_vehicle_implicat']

mapping_columns={'num_incident':'num_incident',
             'Descripcio_sexe':'gender',
             'Edat':'age',
            'Descripcio_tipus_persona':'people_role',
                'Desc_Tipus_vehicle_implicat': 'vehicle',
                'Desc._Tipus_vehicle_implicat': 'vehicle',}
persones=[]

for d in dict_files['persones']:
    #print(d.columns)
    d.columns=[fb.posant_accents(col) for col in d.columns]
    d.columns=[col if not col.startswith('Desc._') else 'Desc_Tipus_vehicle_implicat' for col in d.columns]
    n_columns=[col for col in d.columns if col in chosen_features]
    f=d[n_columns].copy()
    f.columns=[mapping_columns[columna] for columna in f.columns]
    persones.append(f)
    
persones=pd.concat(persones)
persones=persones.reset_index()
persones['num_incident']=[x.strip() for x in persones.num_incident]
#replace also H and D from 2024
persones['gender']=persones['gender'].replace(['Home',"H",'Dona',"D","Desconegut"], [0,0,1,1,np.NaN])
persones['people_role']=persones['people_role'].replace(list(persones.people_role.unique()),['passenger','driver','pedestrian',np.NaN])
persones['age']=persones['age'].replace(['Desconegut',-1],np.NaN)
# persones=persones.replace('Dona',1).replace('Home',0).replace("Conductor",'driver').replace('Vianant', 'pedestrian').replace("Passatger",'passenger').replace('Desconegut',np.NaN).replace(-1,np.NaN)
persones['age']=persones['age'].astype(float)
persones['vehicle']=persones.vehicle.map(fb.map_vehicles)
for ix in persones[(persones.people_role=='driver')&(persones.age==0)].index:
    persones.loc[ix,'age']=np.NaN
for ix in persones[(persones.age>90)&(persones.people_role=='driver')].index:
    persones.loc[ix,'age']=np.NaN
persones['vehicle']=persones.vehicle.fillna('unknown')
persones['vehicle']=['truck' if 'truck' in x else 'bus' if ' bus' in x or x=='tram' else 'car' if x=='suv' else 'van' if 'minibus' in x else x for x in persones.vehicle]
persones['vehicle']=persones.vehicle.replace('unkown',np.NaN)

##TODO: once i fix the age issue based in the vehicle
misc_vehicles=persones.vehicle.value_counts()[persones.vehicle.value_counts()<1000].index
persones['vehicle']=[veh if veh not in misc_vehicles else 'misc_vehicle' for veh in persones.vehicle]

people=persones.set_index('num_incident').drop('index',axis=1)

people=pd.get_dummies(people)

index_1=people.index
vehicle_cols=[col for col in people.columns if 'vehicle_' in col]


scaler = MinMaxScaler()
people = pd.DataFrame(scaler.fit_transform(people), columns = people.columns)


imputer = KNNImputer(n_neighbors=5)
people = pd.DataFrame(imputer.fit_transform(people),columns = people.columns)

people=pd.DataFrame(scaler.inverse_transform(people), columns = people.columns)
vehicle_df=people[vehicle_cols].copy()
#people=people.drop(vehicle_cols,axis=1)
#people.isnull().sum()
people['gender_100/1']=[100 if gen<=0.5 else 1 for gen in people.gender]
people['age_driver']=people.age*people.people_role_driver
people['gender_driver']=people['gender_100/1']*people.people_role_driver
if 'gender' in list(people.columns):
    people=people.drop('gender',axis=1)
people=people.set_index(index_1)
people['driver_u_25_count']=[1 if x < 27 else 0 for x in people.age_driver]
people=people.reset_index().groupby('num_incident').sum()
people['age_driver']=[x[0]/x[1] if (x[1]!=0) else np.nan for x in zip(people['age_driver'],people['people_role_driver'])]
people['gender_driver_male_count']=[int(str(gen)[0]) if len(str(gen))==3 else 0 for gen in people.gender_driver.astype(int)]
people['gender_driver_female_count']=[int(str(gen)[2]) if len(str(gen))==3 else gen for gen in people.gender_driver.astype(int)]
people['age_bins']=pd.cut(people.age_driver,5, labels=range(1,6))
people=people.drop(['age','gender_100/1','gender_driver'],axis=1)
#people.merge()

total= pd.merge(total,people, how='left',on='num_incident')
total['age_driver']=total.age_driver.fillna(total.age_driver.mean())
#print(total.isnull().sum())
people.to_csv(f'./data/people{last_year}.csv', index=False)

### NEXT STEP: What to do with the nulls in total after merging with people?WILL WAIT UNTIL FINAL TOTAL

print("DONE WITH PEOPLE")

###TYPE


tipus=pd.DataFrame(columns=['num_incident','accident_type'])

for df in dict_files['tipus'].copy():
    
    tipus=pd.concat([tipus,df[['num_incident','accident_type']]])
    



type_accident_map={'Atropellament': 'run_over',
         'Col.lisio lateral': 'lateral_collision',
        'Xoc contra element estatic': 'crash_into_stationary',
      'Abast': 'rear-end_collision',
       'Col.lisio frontal':'frontal_collision',
      'Col.lisio fronto-lateral':'frontal-lateral_collision',
      'Caiguda (dues rodes)':'fall--motorcycle',
      'Abast multiple':'multiple_rear-end_collision',
      'Caiguda interior vehicle':'fall_inside_vehicle',
      'Altres':'Other_types',
      'Bolcada (mes de dues rodes)':'overturning',
      'Desconegut':'unknown',
      'Sortida de via amb xoc o col.lisio':'run-off_with_crash_or_collision',
      'Encalç':'rear-end_collision',
      'Sortida de via amb bolcada':'run-off_with_overturning',
      'Xoc amb animal a la calçada':'crash_into_animal_on_road',
      'Resta sortides de via':'run-off_not_included_previously'}
tipus.shape

tipus['accident_type']=tipus['accident_type'].apply(fb.posant_accents).map(type_accident_map)

tipus['num_incident']=[x.strip() for x in tipus.num_incident]
tipus=tipus.dropna()
misc_type=list(tipus.accident_type.value_counts()[tipus.accident_type.value_counts()<350].index)
misc_type.append('Other_types')
tipus=tipus.groupby('num_incident')['accident_type'].agg(lambda x: ','.join(x)).reset_index()
tipus['accident_type']=["misc_type" if x in misc_type else 'frontal' if 'frontal' in x else 'collision' if 'collision' in x else 'crash' if 'crash' in x else 'run-off' if 'run-off' in x else x for x in tipus.accident_type]


type_list=tipus.accident_type.value_counts().index
tipus['accident_type']=[fb.organizing_types(x,type_list) for x in  tipus.accident_type]
#merging misc_type and Other_types
tipus['accident_type']=[name if name !='Other_types' else 'misc_type' for name in tipus.accident_type]
tipus2=tipus.copy()
total4=total.copy()
total= pd.merge(total,tipus, how='left',on='num_incident')
#total.fillna('misc_type',inplace=True)
total5=total.copy()
tipus.to_csv(f'./data/types{last_year}.csv',index=False)

print('type: ', tipus.shape, 'Total: ',total.shape)

print("DONE WITH TYPE")

##VEHICLE***Model no surt a tots els anys. About seniority I am going to replace Desconegut with null
##It is impossible to link people to vehicles in order to guess the value of missing license seniority. I cannot base it in age. It is not possible to link people to vehicle.
columns_to_add=['num_incident', 'vehicle_model', 'vehicle_brand_name',
       'vehicle_color', 'license', 'license_seniority' ]
#vehicles=pd.DataFrame(columns=columns_to_add)
vehicles_list=[]
for df in dict_files['vehicles'].copy():
    vehicles_list.append(df[columns_to_add])
    vehicles=pd.concat([vehicles,df[columns_to_add]])
vehicles =pd.concat(vehicles_list)
vehicles=vehicles.dropna()
vehicles['num_incident']=[x.strip() for x in vehicles.num_incident]
vehicles2=vehicles.copy()
#compiling values to calculate MEDIAN
median_list=[]
for v in vehicles.license_seniority:
    if isinstance(v,float):
        median_list.append(int(v))
    elif v!='Desconegut':
        #print(v)
        median_list.append(int(v))
    
#Calculating MEDIAN
if len(median_list) %2 !=2:
    
    MEDIAN=median_list[len(median_list)//2]
    #print(MEDIAN)
else:
    MEDIAN =median_list[len(median_list)//2]+median_list[len(median_list)//2-1]/2
    
vehicles['license_seniority']=vehicles.license_seniority.replace('Desconegut',MEDIAN).astype(int)
vehicles_seniority=vehicles.groupby('num_incident')[['license_seniority']].mean()


vehicles=vehicles.drop('license_seniority',axis=1).groupby('num_incident').agg(lambda x: ','.join(x))

vehicles=pd.concat([vehicles,vehicles_seniority],axis=1).reset_index()
vehicles.to_csv(f'./data/vehicles{last_year}.csv',index=False)
total= pd.merge(total,vehicles, how='left',on='num_incident')
##I will input the most common one
total=total.apply(lambda x: x.fillna(x.value_counts().index[0]))

total.to_csv(f'./data/accidents{last_year}.csv',index=False)
#total.fillna(-1,inplace=True)
print('vehicles: ', vehicles.shape, 'Total: ',total.shape)


### NEXT STEP: What to do with the nulls in total after merging with people?WILL WAIT UNTIL FINAL TOTAL
url = "https://www.youtube.com/watch?v=Udt-9J8nzGE"
webbrowser.open(url,new=1)
print('DONE')

What is the last year we are doing? 2024


(131495, 18)
num_incident           0
district_code          0
district               0
neighborhood           0
street_name            0
weekday                0
year                   0
month                  0
day                    0
hour                   0
ped_cause              0
num_deaths             0
num_minorly_injured    0
num_severly_injured    0
num_victims            0
num_vehicles           0
utm_y                  0
utm_x                  0
dtype: int64
EL BO: (131495, 18)
num_incident           0
district_code          0
district               0
neighborhood           0
street_name            0
weekday                0
year                   0
month                  0
day                    0
hour                   0
ped_cause              0
num_deaths             0
num_minorly_injured    0
num_severly_injured    0
num_victims            0
num_vehicles           0
utm_y                  0
utm_x                  0
dtype: int64
DONE WITH ACCIDENTS
(131495, 18) (133854,

In [202]:
with open("./data/dataframes_dict_files.pkl", "rb" ) as f:
    mydict=pickle.load(f)
mydict.keys()

dict_keys(['causes', 'persones', 'tipus', 'vehicles', '', 'causa_conductor'])

In [223]:
mydict[''][0].isnull().sum()

Numero_expedient                0
Codi_districte                  0
Nom_districte                   0
Codi_barri                      0
Nom_barri                       0
Codi_carrer                     0
Nom_carrer                      0
Num_postal                   2160
Descripcio_dia_setmana          0
NK_Any                          0
Mes_any                         0
Nom_mes                         0
Dia_mes                         0
Hora_dia                        0
Descripcio_torn                 0
Descripcio_causa_vianant     4843
Numero_morts                 5150
Numero_lesionats_lleus        691
Numero_lesionats_greus       4994
Numero_victimes               559
Numero_vehicles_implicats       0
Coordenada_UTM_Y_ED50           0
Coordenada_UTM_X_ED50           0
Longitud_WGS84                  0
Latitud_WGS84                   0
dtype: int64

In [225]:
mydict[''][0].sample(4)

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,NK_Any,...,Descripcio_causa_vianant,Numero_morts,Numero_lesionats_lleus,Numero_lesionats_greus,Numero_victimes,Numero_vehicles_implicats,Coordenada_UTM_Y_ED50,Coordenada_UTM_X_ED50,Longitud_WGS84,Latitud_WGS84
4785,2024S004800,10,Sant Martí,66,el Parc i la Llacuna del Poblenou,286504,Roger de Flor,NaN,Dimarts,2024,...,NaN,NaN,1.0,NaN,1.0,2,431736.825,4582736.655,2.182353,41.391419
832,2024S000836,3,Sants-Montjuïc,12,la Marina del Prat Vermell,700134,Transversal 8,16,Dimarts,2024,...,NaN,NaN,NaN,NaN,NaN,1,426028.646,4576165.020,2.114884,41.331726
1505,2024S001510,10,Sant Martí,66,el Parc i la Llacuna del Poblenou,30500,Badajoz,NaN,Dijous,2024,...,NaN,NaN,1.0,NaN,1.0,2,432609.473,4583758.694,2.192676,41.400698
4226,2024S004241,3,Sants-Montjuïc,13,la Marina de Port,369408,Zona Franca,123-133,Dijous,2024,...,NaN,NaN,1.0,NaN,1.0,2,428200.170,4579031.230,2.140493,41.357738
